# Feladatok generáslása
szöveggeneráló modellek tanítására

In [ ]:
from datasets import load_dataset, load_from_disk

In [ ]:
dataset = load_dataset('boapps/kmdb_classification')

In [ ]:
dataset

## Cikk klasszifikációja

In [ ]:
article_classification_prompt = '''[korrupció klasszifikáció]
{title}

{description}

{text}

cimkék: {keywords}

###

téma: {theme}
'''

In [ ]:
article_classification_prompt = '''[korrupció klasszifikáció]
{title}

{description}

cimkék: {keywords}

###

téma: {theme}
'''

In [ ]:
def generate_article_classification_task(article):
    article['keywords'] = ', '.join(article['keywords'])
    article['theme'] = ['egyéb', 'korrupció'][article['label']]
    article['description'] = article['description'].replace(article['title'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'].replace(article['description'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'][:article['text'].find('\n')]
    return {'text': article_classification_prompt.format(**article), 'type': 'article_classification'}

    k = 0
print(generate_article_classification_task(dataset['train'][1])['text'])

In [ ]:
# TODO ez szebb lenne format string-el, de akkor 20000. iteracional kicrashel valami key errorral, ami egy windowsos registry keynek tunik, WTF (linuxon vagyok)
def generate_article_classification_task_b(articles):
    results = {'text': [], 'label': [], 'type': []}
    for i in range(len(articles['text'])):
        articles['keywords'][i] = ', '.join(articles['keywords'][i])
        articles['description'][i] = articles['description'][i].replace(articles['title'][i], '').replace(' - ', ' ').strip()
        articles['text'][i] = articles['text'][i].replace(articles['description'][i], '').replace(' - ', ' ').strip()
        n = 0
        for paragraph in articles['text'][i].splitlines():
            if paragraph.strip() == articles['description'][i]:
                continue
            if n == 5:
                break
            if len(paragraph.strip()) > 250:
                n += 1
                text = '''[korrupció klasszifikáció]
'''+articles['title'][i]+'''

'''+articles['description'][i]+'''

'''+paragraph+'''

cimkék: '''+articles['keywords'][i]+'''

###

téma: '''+['egyéb', 'korrupció'][articles['label'][i]]+'''
'''
                results['text'].append(text)
                results['type'].append('article_classification')
                results['label'].append(articles['label'][i])
    return results

#print(generate_article_classification_task(dataset['train'][1])['text'])

In [ ]:
article_classification_task_dataset = dataset.map(generate_article_classification_task, remove_columns=['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'])

In [ ]:
article_classification_task_dataset = article_classification_task_dataset #.remove_columns('theme')

In [ ]:
print(article_classification_task_dataset['train'][1]['text'])

In [ ]:
article_classification_task_dataset.save_to_disk('../datasets/tasks/article_classification_task_dataset')

## Releváns személyek adathalmaz

In [ ]:
entity_dataset = load_from_disk('../datasets/intermediate/entity_dataset')

In [ ]:
def make_people_dataset(articles):
    results = {'text': [], 'corrupt_people': [], 'not_corrupt_people': []}
    for i in range(len(articles['text'])):
        if articles['entities'][i] is None:
            continue
        entities = [tuple(entity) for entity in articles['entities'][i]]
        if articles['kmonitor_people'][i]:
            has_missing = False
            people = [p if ' (' not in p else p[:p.find(' (')] for p in articles['kmonitor_people'][i]]
            corrupt_people = set()
            not_corrupt_people = set()
            for person in people:
                isin = False
                original_name = ''
                for ent, label in entities:
                    if person.lower().replace('-', '').replace('dr. ', '') == ent.lower().replace('-', '').replace('dr. ', ''):
                        isin = True
                        original_name = ent
                        break
                if not isin:
                    if not has_missing:
                        print(i, articles['title'][i])
                        print(articles['url'][i])
                    print('-', person, 'not in')
                    has_missing = True
                else:
                    corrupt_people.add(original_name)
            for ent, label in entities:
                correct = True
                for person in people:
                    if ent.lower().replace('-', '').replace('dr. ', '') in person.lower().replace('-', '').replace('dr. ', '') or label != 'PER' or ' ' not in ent:
                        correct = False
                        break
                if correct:
                    not_corrupt_people.add(ent)
            if not has_missing:
                results['text'].append(articles["text"][i])
                results['corrupt_people'].append(list(corrupt_people))
                results['not_corrupt_people'].append(list(not_corrupt_people))
    return results

In [ ]:
people_dataset = entity_dataset.map(make_people_dataset, batched=True, remove_columns=dataset['train'].column_names+['entities'])

In [ ]:
people_dataset

In [ ]:
people_dataset.save_to_disk('../datasets/intermediate/people_dataset')

## Személy-klasszifikációs feladatok generálása

In [ ]:
people_dataset = load_from_disk('../datasets/intermediate/people_dataset')

In [ ]:
people_classification_prompt = '''[személy klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett: {corrupt}
'''

In [ ]:
def generate_people_classification_task(articles):
    result = {'text': [], 'type': []}
    for i in range(len(articles['text'])):
        article = {key: articles[key][i] for key in articles}
        article['corrupt'] = ', '.join(sorted(article['corrupt_people']))
        article['other'] = ', '.join(sorted(article['not_corrupt_people']))
        article['all'] = ', '.join(sorted(article['corrupt_people'] + article['not_corrupt_people']))
        result['type'].append('people_classification')
        result['text'].append(people_classification_prompt.format(**article))
    return result

In [ ]:
people_classification_task_dataset = people_dataset.map(generate_people_classification_task, batched=True, remove_columns=people_dataset['train'].column_names)

In [ ]:
print(people_classification_task_dataset['train'][0]['text'])

In [ ]:
people_classification_task_dataset.save_to_disk('../datasets/tasks/people_classification_task_dataset')

## Intézmény klasszifikációja

In [ ]:
from datasets import load_from_disk
entity_dataset = load_from_disk('../datasets/intermediate/entity_dataset')

In [ ]:
set([tuple(entity) for entities in entity_dataset['train']['entities'][:10] for entity in entities if entity[1] == 'ORG'])

In [ ]:
def make_institutions_dataset(articles):
    results = {'text': [], 'corrupt_institutions': [], 'not_corrupt_institutions': []}
    for i in range(len(articles['text'])):
        if articles['entities'][i] is None:
            continue
        entities = [tuple(entity) for entity in articles['entities'][i]]
        if articles['kmonitor_institutes'][i]:
            has_missing = False
            institutions = [p if ' (' not in p else p[:p.find(' (')] for p in articles['kmonitor_institutes'][i]]
            corrupt_institutions = set()
            not_corrupt_institutions = set()
            for institution in institutions:
                isin = False
                original_name = ''
                for ent, label in entities:
                    if institution.lower().replace(' zrt.', '').replace(' kft.', '') == ent.lower().replace(' zrt.', '').replace(' kft.', ''):
                        isin = True
                        original_name = ent
                        break
                if not isin:
                    if not has_missing:
                        print(i, articles['title'][i])
                        print(articles['url'][i])
                    print('-', institution, 'not in')
                    has_missing = True
                else:
                    corrupt_institutions.add(original_name)
            for ent, label in entities:
                correct = True
                for institution in institutions:
                    if ent.lower().replace(' zrt.', '').replace(' kft.', '') in institution.lower().replace(' zrt.', '').replace(' kft.', '') or label != 'ORG' or ' ' not in ent:
                        correct = False
                        break
                if correct:
                    not_corrupt_institutions.add(ent)
            if not has_missing:
                results['text'].append(articles["text"][i])
                results['corrupt_institutions'].append(list(corrupt_institutions))
                results['not_corrupt_institutions'].append(list(not_corrupt_institutions))
    return results

In [ ]:
institutions_dataset = entity_dataset.map(make_institutions_dataset, batched=True, remove_columns=dataset['train'].column_names+['entities'])

In [ ]:
institutions_dataset

In [ ]:
institutions_dataset.save_to_disk('../datasets/intermediate/institutions_dataset')

## Intézményklasszifikációs feladatok generálása

In [ ]:
institutions_dataset = load_from_disk('../datasets/intermediate/institutions_dataset')

In [ ]:
institution_classification_prompt = '''[intézmény klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett: {corrupt}
'''

In [ ]:
def generate_institution_classification_task(articles):
    result = {'text': [], 'type': []}
    for i in range(len(articles['text'])):
        article = {key: articles[key][i] for key in articles}
        article['corrupt'] = ', '.join(sorted(article['corrupt_institutions']))
        article['other'] = ', '.join(sorted(article['not_corrupt_institutions']))
        article['all'] = ', '.join(sorted(article['corrupt_institutions'] + article['not_corrupt_institutions']))
        result['type'].append('institution_classification')
        result['text'].append(people_classification_prompt.format(**article))
    return result

In [ ]:
institution_classification_task_dataset = institutions_dataset.map(generate_institution_classification_task, batched=True, remove_columns=institutions_dataset['train'].column_names)

In [ ]:
institution_classification_task_dataset

In [ ]:
institution_classification_task_dataset.save_to_disk('../datasets/tasks/institution_classification_task_dataset')

## Helyszínek klasszifikációja

In [ ]:
set([tuple(entity) for entities in entity_dataset['train']['entities'][:10] for entity in entities if entity[1] == 'LOC'])

In [ ]:
set([entity for entities in entity_dataset['train']['kmonitor_places'][:10] for entity in entities])

## Címkék generálása

In [ ]:
keyword_dataset = dataset.filter(lambda article: article['label'] == 1)

In [ ]:
keyword_generation_prompt = '''[címkék generálása]
{title}

{description}

{text}

cimkék: {keywords}

###

korrupciós címkék: {kmonitor_tags}
'''

In [ ]:
def generate_keyword_generation_task(article):
    article['keywords'] = ', '.join(article['keywords'])
    article['kmonitor_tags'] = ', '.join(article['kmonitor_tags'])
    return {'text': keyword_generation_prompt.format(**article), 'type': 'keyword_generation'}

print(generate_keyword_generation_task(keyword_dataset['train'][1])['text'])

In [ ]:
keyword_generation_task_dataset = keyword_dataset.map(generate_keyword_generation_task, remove_columns=keyword_dataset['train'].column_names)

In [ ]:
keyword_generation_task_dataset

In [ ]:
keyword_generation_task_dataset.save_to_disk('../datasets/tasks/keyword_generation_task_dataset')

## helyreigazítás detektálása

## ügyek detektálása

## kapcsolatok ent

## Összes feladat

In [ ]:
article_classification_task_dataset = load_from_disk('../datasets/tasks/article_classification_task_dataset/')
institution_classification_task_dataset = load_from_disk('../datasets/tasks/institution_classification_task_dataset//')
keyword_generation_task_dataset = load_from_disk('../datasets/tasks/keyword_generation_task_dataset/')
people_classification_task_dataset = load_from_disk('../datasets/tasks/people_classification_task_dataset//')

In [ ]:
article_classification_task_dataset